In [1]:
import chardet
import pandas as pd

In [153]:
file_path = "./data/base/Health Examination Information_2023.CSV"

# 1. 인코딩 자동 감지
with open(file_path, 'rb') as f:
    rawdata = f.read(10000)

encoding_detected = chardet.detect(rawdata)['encoding']
print(f"파일 인코딩 감지: {encoding_detected}")

파일 인코딩 감지: EUC-KR


In [154]:
# 2. 감지된 인코딩으로 파일 불러오기
df = pd.read_csv(file_path, encoding=encoding_detected)

In [155]:
df.head()

,기준년도,가입자일련번호,시도코드,성별코드,연령대코드(5세단위),신장(5cm단위),체중(5kg단위),허리둘레,시력(좌),시력(우),...,혈청지피티(ALT),감마지티피,흡연상태,음주여부,구강검진수검여부,치아우식증유무,결손치 유무,치아마모증유무,제3대구치(사랑니) 이상,치석
0,2023,34735,46,2,9,155,70,92.0,1.2,1.2,...,24.0,50.0,1.0,1.0,0,NaN,NaN,NaN,NaN,NaN
1,2023,4105118,11,1,17,160,55,86.0,0.9,9.9,...,11.0,31.0,1.0,1.0,0,NaN,NaN,NaN,NaN,NaN
2,2023,362482,36,2,13,150,65,96.0,1.0,0.8,...,29.0,24.0,1.0,1.0,0,NaN,NaN,NaN,NaN,NaN
3,2023,653166,11,1,13,160,70,85.0,1.0,1.2,...,21.0,27.0,3.0,1.0,1,0.0,NaN,NaN,NaN,0.0
4,2023,4152237,41,1,12,165,65,84.5,1.0,1.2,...,33.0,49.0,3.0,1.0,0,NaN,NaN,NaN,NaN,NaN


In [156]:
df.columns

Index(['기준년도', '가입자일련번호', '시도코드', '성별코드', '연령대코드(5세단위)', '신장(5cm단위)',
       '체중(5kg단위)', '허리둘레', '시력(좌)', '시력(우)', '청력(좌)', '청력(우)', '수축기혈압',
       '이완기혈압', '식전혈당(공복혈당)', '총콜레스테롤', '트리글리세라이드', 'HDL콜레스테롤', 'LDL콜레스테롤',
       '혈색소', '요단백', '혈청크레아티닌', '혈청지오티(AST)', '혈청지피티(ALT)', '감마지티피', '흡연상태',
       '음주여부', '구강검진수검여부', '치아우식증유무', '결손치 유무', '치아마모증유무', '제3대구치(사랑니) 이상',
       '치석'],
      dtype='object')

In [157]:
# 필요한 컬럼 리스트를 정의합니다.
needed_columns = [
    '성별코드', '연령대코드(5세단위)', '신장(5cm단위)', '체중(5kg단위)', '허리둘레',
    '수축기혈압', '이완기혈압', '식전혈당(공복혈당)',
    '총콜레스테롤', 'HDL콜레스테롤', 'LDL콜레스테롤', '트리글리세라이드',
    '혈청지오티(AST)', '혈청지피티(ALT)', '감마지티피',
    '흡연상태', '음주여부'
]

In [158]:
df = df[needed_columns]

In [159]:
#  새 CSV 파일로 저장

df.to_csv("./data/health_data.csv", index=False)

In [160]:
df.isna().sum()

성별코드                0
연령대코드(5세단위)         0
신장(5cm단위)           0
체중(5kg단위)           0
허리둘레              411
수축기혈압            5747
이완기혈압            5747
식전혈당(공복혈당)       5814
총콜레스테롤         661394
HDL콜레스테롤       661394
LDL콜레스테롤       667247
트리글리세라이드       661394
혈청지오티(AST)       5816
혈청지피티(ALT)       5816
감마지티피            5813
흡연상태               89
음주여부               56
dtype: int64

In [161]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [162]:
# 다중대체법 적용
imputer = IterativeImputer(random_state=42)
cols = ['총콜레스테롤', 'HDL콜레스테롤', 'LDL콜레스테롤']
df[cols] = imputer.fit_transform(df[cols])

In [163]:
# 중앙값으로 대체(허리둘레, 수축기혈압, 이완기혈압, 식전혈당, 간 기능 관련 변수)
df['트리글리세라이드'] = df['트리글리세라이드'].fillna(df['트리글리세라이드'].median())
df['허리둘레'] = df['허리둘레'].fillna(df['허리둘레'].median())
df['수축기혈압'] = df['수축기혈압'].fillna(df['수축기혈압'].median())
df['이완기혈압'] = df['이완기혈압'].fillna(df['이완기혈압'].median())
df['식전혈당(공복혈당)'] = df['식전혈당(공복혈당)'].fillna(df['식전혈당(공복혈당)'].median())
df['혈청지오티(AST)'] = df['혈청지오티(AST)'].fillna(df['혈청지오티(AST)'].median())
df['혈청지피티(ALT)'] = df['혈청지피티(ALT)'].fillna(df['혈청지피티(ALT)'].median())
df['감마지티피'] = df['감마지티피'].fillna(df['감마지티피'].median())

In [164]:
# 평균치로 결측치 대체 (수축기혈압, 이완기혈압)
df['수축기혈압'] = df['수축기혈압'].fillna(df['수축기혈압'].mean())
df['이완기혈압'] = df['이완기혈압'].fillna(df['이완기혈압'].mean())

In [165]:
# 최빈값으로 결측치 대체 (흡연상태, 음주여부와 같은 범주형 변수
df['흡연상태'] = df['흡연상태'].fillna(df['흡연상태'].mode()[0])
df['음주여부'] = df['음주여부'].fillna(df['음주여부'].mode()[0])

In [166]:
df.isna().sum()

성별코드           0
연령대코드(5세단위)    0
신장(5cm단위)      0
체중(5kg단위)      0
허리둘레           0
수축기혈압          0
이완기혈압          0
식전혈당(공복혈당)     0
총콜레스테롤         0
HDL콜레스테롤       0
LDL콜레스테롤       0
트리글리세라이드       0
혈청지오티(AST)     0
혈청지피티(ALT)     0
감마지티피          0
흡연상태           0
음주여부           0
dtype: int64

In [167]:
# 이상치 탐지 및 처리: 비정상적인 검진 결과 값 확인 및 조정

In [168]:
# 각 변수의 허용 범위를 딕셔너리로 정의
variable_ranges = {
    '총콜레스테롤': (100, 400),
    'HDL콜레스테롤': (20, 100),
    'LDL콜레스테롤': (30, 250),
    '트리글리세라이드': (30, 500),
    '수축기혈압': (70, 250),
    '이완기혈압': (40, 150),
    '식전혈당(공복혈당)': (50, 300),
    '혈청지오티(AST)': (5, 300),
    '혈청지피티(ALT)': (5, 300),
    '감마지티피': (5, 300),
    '신장(5cm단위)': (140, 200),
    '체중(5kg단위)': (30, 150),
    '허리둘레': (50, 150)
}


In [169]:
# 반복문을 사용하여 이상치 처리
for variable, (min_value, max_value) in variable_ranges.items():
    df = df[(df[variable] >= min_value) & (df[variable] <= max_value)]

In [170]:
df

,성별코드,연령대코드(5세단위),신장(5cm단위),체중(5kg단위),허리둘레,수축기혈압,이완기혈압,식전혈당(공복혈당),총콜레스테롤,HDL콜레스테롤,LDL콜레스테롤,트리글리세라이드,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부
0,2,9,155,70,92.0,139.0,89.0,98.0,168.000000,46.000000,86.000000,181.0,26.0,24.0,50.0,1.0,1.0
1,1,17,160,55,86.0,123.0,52.0,87.0,196.744106,57.035215,115.281692,105.0,22.0,11.0,31.0,1.0,1.0
2,2,13,150,65,96.0,109.0,69.0,114.0,220.000000,52.000000,107.000000,307.0,22.0,29.0,24.0,1.0,1.0
4,1,12,165,65,84.5,137.0,86.0,105.0,196.744106,57.035215,115.281692,105.0,23.0,33.0,49.0,3.0,1.0
5,2,5,170,50,69.2,92.0,60.0,93.0,196.744106,57.035215,115.281692,105.0,16.0,11.0,12.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1,8,170,65,78.0,107.0,61.0,83.0,196.744106,57.035215,115.281692,105.0,19.0,13.0,22.0,1.0,1.0
999996,1,10,165,80,96.1,139.0,86.0,118.0,246.000000,59.000000,126.000000,303.0,70.0,65.0,160.0,2.0,1.0
999997,2,11,155,65,87.0,146.0,83.0,156.0,196.744106,57.035215,115.281692,105.0,20.0,26.0,25.0,1.0,1.0
999998,2,7,160,55,69.0,122.0,71.0,105.0,196.744106,57.035215,115.281692,105.0,22.0,20.0,16.0,1.0,0.0


In [171]:
# BMI(체질량지수)
df['BMI'] = df['체중(5kg단위)'] / (df['신장(5cm단위)'] / 100) ** 2

In [172]:
# 신장(5cm단위),체중(5kg단위) 컬럼제외 

In [173]:
df = df.drop(['신장(5cm단위)', '체중(5kg단위)'], axis=1)

In [174]:
df

,성별코드,연령대코드(5세단위),허리둘레,수축기혈압,이완기혈압,식전혈당(공복혈당),총콜레스테롤,HDL콜레스테롤,LDL콜레스테롤,트리글리세라이드,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,BMI
0,2,9,92.0,139.0,89.0,98.0,168.000000,46.000000,86.000000,181.0,26.0,24.0,50.0,1.0,1.0,29.136316
1,1,17,86.0,123.0,52.0,87.0,196.744106,57.035215,115.281692,105.0,22.0,11.0,31.0,1.0,1.0,21.484375
2,2,13,96.0,109.0,69.0,114.0,220.000000,52.000000,107.000000,307.0,22.0,29.0,24.0,1.0,1.0,28.888889
4,1,12,84.5,137.0,86.0,105.0,196.744106,57.035215,115.281692,105.0,23.0,33.0,49.0,3.0,1.0,23.875115
5,2,5,69.2,92.0,60.0,93.0,196.744106,57.035215,115.281692,105.0,16.0,11.0,12.0,1.0,1.0,17.301038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1,8,78.0,107.0,61.0,83.0,196.744106,57.035215,115.281692,105.0,19.0,13.0,22.0,1.0,1.0,22.491349
999996,1,10,96.1,139.0,86.0,118.0,246.000000,59.000000,126.000000,303.0,70.0,65.0,160.0,2.0,1.0,29.384757
999997,2,11,87.0,146.0,83.0,156.0,196.744106,57.035215,115.281692,105.0,20.0,26.0,25.0,1.0,1.0,27.055151
999998,2,7,69.0,122.0,71.0,105.0,196.744106,57.035215,115.281692,105.0,22.0,20.0,16.0,1.0,0.0,21.484375


In [175]:
#  새 CSV 파일로 저장
df.to_csv('bmi_data.csv', index=False)

In [44]:
X_health = df

In [115]:
X_health

,성별코드,연령대코드(5세단위),허리둘레,수축기혈압,이완기혈압,식전혈당(공복혈당),총콜레스테롤,HDL콜레스테롤,LDL콜레스테롤,트리글리세라이드,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,BMI
0,2,9,92.0,139.0,89.0,98.0,168.000000,46.000000,86.000000,181.0,26.0,24.0,50.0,1.0,1.0,29.136316
1,1,17,86.0,123.0,52.0,87.0,196.744106,57.035215,115.281692,105.0,22.0,11.0,31.0,1.0,1.0,21.484375
2,2,13,96.0,109.0,69.0,114.0,220.000000,52.000000,107.000000,307.0,22.0,29.0,24.0,1.0,1.0,28.888889
4,1,12,84.5,137.0,86.0,105.0,196.744106,57.035215,115.281692,105.0,23.0,33.0,49.0,3.0,1.0,23.875115
5,2,5,69.2,92.0,60.0,93.0,196.744106,57.035215,115.281692,105.0,16.0,11.0,12.0,1.0,1.0,17.301038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1,8,78.0,107.0,61.0,83.0,196.744106,57.035215,115.281692,105.0,19.0,13.0,22.0,1.0,1.0,22.491349
999996,1,10,96.1,139.0,86.0,118.0,246.000000,59.000000,126.000000,303.0,70.0,65.0,160.0,2.0,1.0,29.384757
999997,2,11,87.0,146.0,83.0,156.0,196.744106,57.035215,115.281692,105.0,20.0,26.0,25.0,1.0,1.0,27.055151
999998,2,7,69.0,122.0,71.0,105.0,196.744106,57.035215,115.281692,105.0,22.0,20.0,16.0,1.0,0.0,21.484375


In [48]:
import glob

In [4]:
food_data = pd.read_csv('./data/food_data.csv')


In [129]:
# 파일 목록 가져오기
file_list = glob.glob(file_pattern)

In [130]:
# 데이터프레임 목록 초기화
df_list = []

In [131]:
# 모든 파일을 순회하며 데이터프레임에 추가
for file_path in file_list:
    df = pd.read_csv(file_path)
    df_list.append(df)

In [134]:
# 모든 데이터프레임 병합
df_food = pd.concat(df_list, ignore_index=True)

In [135]:
df_food

,Unnamed: 0.1,Unnamed: 0,food,Caloric Value,Fat,Saturated Fats,Monounsaturated Fats,Polyunsaturated Fats,Carbohydrates,Sugars,...,Calcium,Copper,Iron,Magnesium,Manganese,Phosphorus,Potassium,Selenium,Zinc,Nutrition Density
0,0,0,cream cheese,51,5.0,2.900,1.300,0.200,0.8,0.500,...,0.008,14.100,0.082,0.027,1.300,0.091,15.5,19.100,0.039,7.070
1,1,1,neufchatel cheese,215,19.4,10.900,4.900,0.800,3.1,2.700,...,99.500,0.034,0.100,8.500,0.088,117.300,129.2,0.054,0.700,130.100
2,2,2,requeijao cremoso light catupiry,49,3.6,2.300,0.900,0.000,0.9,3.400,...,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,5.400
3,3,3,ricotta cheese,30,2.0,1.300,0.500,0.002,1.5,0.091,...,0.097,41.200,0.097,0.096,4.000,0.024,30.8,43.800,0.035,5.196
4,4,4,cream cheese low fat,30,2.3,1.400,0.600,0.042,1.2,0.900,...,22.200,0.072,0.008,1.200,0.098,22.800,37.1,0.034,0.053,27.007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,227,227,muesli master crumble,124,2.4,1.000,19.800,8.500,4.5,0.042,...,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,9.300
2391,228,228,bran flakes,131,0.8,0.200,0.100,0.500,32.2,7.400,...,0.025,17.600,0.300,11.200,91.600,1.400,179.6,213.200,0.083,45.358
2392,229,229,nut cereal,245,5.9,1.000,3.900,0.500,46.2,17.200,...,75.600,0.200,10.300,39.000,1.000,106.800,134.4,0.066,3.000,156.500
2393,230,230,corn flakes,108,0.3,0.098,0.071,0.018,24.6,2.200,...,0.000,0.600,0.083,5.400,2.000,0.060,9.2,30.000,0.077,27.947


In [141]:
# 필요한 컬럼 선택
food_features = ['food','Caloric Value', 'Fat', 'Saturated Fats', 'Monounsaturated Fats',
                 'Polyunsaturated Fats', 'Carbohydrates', 'Sugars', 'Protein',
                 'Dietary Fiber', 'Cholesterol', 'Sodium', 'Water']

In [142]:
X_food = df_food[food_features]

In [143]:
X_food = X_food.rename(columns={
    'food': '음식',
    'Caloric Value': '칼로리',
    'Fat': '지방',
    'Saturated Fats': '포화지방',
    'Monounsaturated Fats': '단일불포화지방',
    'Polyunsaturated Fats': '다중불포화지방',
    'Carbohydrates': '탄수화물',
    'Sugars': '당류',
    'Protein': '단백질',
    'Dietary Fiber': '식이섬유',
    'Cholesterol': '콜레스테롤',
    'Sodium': '나트륨',
    'Water': '수분'
})

In [144]:
df_food = X_food

In [145]:
def fill_missing_values(df_food):
    df['칼로리'] = df['칼로리'].fillna(df['칼로리'].median())
    df['지방'] = df['지방'].fillna(df['지방'].median())
    df['포화지방'] = df['포화지방'].fillna(df['포화지방'].median())
    df['단일불포화지방'] = df['단일불포화지방'].fillna(df['단일불포화지방'].median())
    df['다중불포화지방'] = df['다중불포화지방'].fillna(df['다중불포화지방'].median())
    df['탄수화물'] = df['탄수화물'].fillna(df['탄수화물'].median())
    df['당류'] = df['당류'].fillna(df['당류'].median())
    df['단백질'] = df['단백질'].fillna(df['단백질'].median())
    df['식이섬유'] = df['식이섬유'].fillna(0)
    df['콜레스테롤'] = df['콜레스테롤'].fillna(df['콜레스테롤'].median())
    df['나트륨'] = df['나트륨'].fillna(df['나트륨'].median())
    df['수분'] = df['수분'].fillna(df['수분'].mean())
    return df_food

In [146]:
df_food

,음식,칼로리,지방,포화지방,단일불포화지방,다중불포화지방,탄수화물,당류,단백질,식이섬유,콜레스테롤,나트륨,수분
0,cream cheese,51,5.0,2.900,1.300,0.200,0.8,0.500,0.9,0.0,14.6,0.016,7.6
1,neufchatel cheese,215,19.4,10.900,4.900,0.800,3.1,2.700,7.8,0.0,62.9,0.300,53.6
2,requeijao cremoso light catupiry,49,3.6,2.300,0.900,0.000,0.9,3.400,0.8,0.1,0.0,0.000,0.0
3,ricotta cheese,30,2.0,1.300,0.500,0.002,1.5,0.091,1.5,0.0,9.8,0.017,14.7
4,cream cheese low fat,30,2.3,1.400,0.600,0.042,1.2,0.900,1.2,0.0,8.1,0.046,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,muesli master crumble,124,2.4,1.000,19.800,8.500,4.5,0.042,2.4,0.0,0.0,0.000,0.0
2391,bran flakes,131,0.8,0.200,0.100,0.500,32.2,7.400,4.0,7.3,0.0,0.200,1.4
2392,nut cereal,245,5.9,1.000,3.900,0.500,46.2,17.200,4.0,3.6,0.0,0.300,2.6
2393,corn flakes,108,0.3,0.098,0.071,0.018,24.6,2.200,1.7,0.8,0.0,0.200,0.9


In [147]:
df_food.isnull().sum()

음식         0
칼로리        0
지방         0
포화지방       0
단일불포화지방    0
다중불포화지방    0
탄수화물       0
당류         0
단백질        0
식이섬유       0
콜레스테롤      0
나트륨        0
수분         0
dtype: int64

In [152]:
df_food.to_csv("./data/food_data.csv", index=False)

In [110]:

file_path = "./data/exercise.csv"

# 1. 인코딩 자동 감지
with open(file_path, 'rb') as f:
    rawdata = f.read(10000)

encoding_detected = chardet.detect(rawdata)['encoding']
print(f"파일 인코딩 감지: {encoding_detected}")

파일 인코딩 감지: EUC-KR


In [111]:
# 2. 감지된 인코딩으로 파일 불러오기
df_ex= pd.read_csv(file_path, encoding=encoding_detected)

In [112]:
df_ex

,운동명,단위체중당에너지소비량
0,바벨 스쿼트,6.0
1,다트,2.5
2,야구 캐치볼,2.5
3,당구,2.5
4,요가,2.5
...,...,...
370,새천년체조,3.5
371,훌라후프,4.0
372,번지피지오,5.0
373,wty운동,7.0


In [10]:
df = pd.read_csv('predictions.csv')

In [11]:
df.isna().sum()

user_id                 88
prediction_exercise    113
prediction_food        113
timestamp              113
이름                       0
성별                       0
연령대                     54
BMI                     23
현재 체중                   59
목표 체중                   60
운동 가능성                  14
운동 확률                    0
식단 개선 필요성               14
식단 확률                    0
나이                     104
현재 체중 (kg)              66
목표체중 (kg)              112
목표 체중 (kg)              67
연령대코드(5세단위)             77
예측 날짜                   88
허리둘레 (cm)              110
dtype: int64

In [6]:
# 2. 결측치 처리 함수 정의
def handle_missing_values(df):
    # 2.1. 텍스트 데이터 처리 ("🔍 예측을 실행하세요" -> "미실행")
    text_cols = ['운동 가능성', '식단 개선 필요성']
    for col in text_cols:
        df[col] = df[col].replace(['🔍 예측을 실행하세요'], '미실행')

In [8]:
    # 2.2. 수치형 데이터 처리 ("미입력" -> NaN -> 중앙값 대체)
numeric_cols = ['연령대', '현재 체중', '목표 체중', '연령대코드(5세단위)', '허리둘레 (cm)']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].fillna(df[col].median())

,user_id,prediction_exercise,prediction_food,timestamp,이름,성별,연령대,BMI,현재 체중,목표 체중,...,운동 확률,식단 개선 필요성,식단 확률,나이,현재 체중 (kg),목표체중 (kg),목표 체중 (kg),연령대코드(5세단위),예측 날짜,허리둘레 (cm)
